In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Importing neccessary libraries for data manipuation and data visualization

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join

In [ ]:
#Setting the dimensions of all the images to 64*64
img_width = 64
img_height = 64

train_data_dir = '/content/drive/MyDrive/TrainingSet/Old_DataSet _PP/Old_Classification/Train'
validation_data_dir = '/content/drive/MyDrive/TrainingSet/Old_DataSet _PP/Old_Classification/Validation'

train_samples = 30
validation_samples = 15
epochs = 100
batch_size = 4          # Model generation in each epoch in batch size of 4 

# Checking for TensorFlow
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
# Conv2D : Two dimenstional convulational model.
# 32 : Input for next layer
# (3,3) convulonational windows size

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # Output convert into one dimension layer and will go to Dense layer
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
import keras
from keras import optimizers
model.compile(loss='binary_crossentropy', 
              optimizer=keras.optimizers.Adam(lr=.0001),
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

#normal + cancer in train

In [ ]:
print(train_generator.class_indices)

In [ ]:
imgs, labels = next(train_generator)

In [ ]:
from skimage import io

def imshow(image_RGB):
  io.imshow(image_RGB)
  io.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
image_batch,label_batch = train_generator.next()

print(len(image_batch))
for i in range(0,len(image_batch)):
    image = image_batch[i]
    print(label_batch[i])
    imshow(image)

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
# history = model.fit_generator(
#     train_generator,
#     steps_per_epoch=train_samples // batch_size,
#     epochs=epochs,
#     validation_data=validation_generator,
#     validation_steps=validation_samples // batch_size)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples)

In [ ]:
train_acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
val_acc=val_acc[28]

t=max(train_acc)*100
v=val_acc*100
print("Training Accuracy: %.1f"%t)
print("Validation Accuracy: %.1f"%v)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
predict_dir_path='/content/drive/MyDrive/TrainingSet/Old_DataSet _PP/Old_Classification/Test/'
onlyfiles = [f for f in listdir(predict_dir_path) if isfile(join(predict_dir_path, f))]
print(onlyfiles)

In [ ]:
# predicting images
from keras.preprocessing import image
cancer_counter = 0 
normal_counter  = 0
for file in onlyfiles:
    img = image.load_img(predict_dir_path+file, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0:
        print(file + ": " + 'cancer')
        cancer_counter += 1
    else:
        print(file + ": " + 'normal')
        normal_counter += 1
print("Total Cancer :",cancer_counter)
print("Total Normal :",normal_counter)

In [ ]:
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from keras.preprocessing import image

In [ ]:
path='/content/drive/MyDrive/TrainingSet/Old_DataSet _PP/Old_Classification/Test/14.jpg'
img = image.load_img(path , target_size =(64,64))
x = image.img_to_array(img)
plt.imshow(x/255.)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images , batch_size = 2)
#print(classes)
if classes[0] <0.5 :
  print("Cancerous Tumor")
else:
  print("Benign Tumor")

In [ ]:
path='/content/drive/MyDrive/TrainingSet/Old_DataSet _PP/Old_Classification/Test/n23.jpg'
img = image.load_img(path , target_size =(64,64))
x = image.img_to_array(img)
plt.imshow(x/255.)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images , batch_size = 2)
#print(classes)
if classes[0] <0.5 :
  print("Cancerous Tumor")
else:
  print("Benign Tumor")

In [ ]:
# %matplotlib inline
import tensorflow as tf
import numpy as np
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

preds = model.predict(validation_generator,verbose=1)


In [ ]:
fpr , tpr , _ = roc_curve(validation_generator.classes , preds)
roc_auc = auc (fpr , tpr)



In [ ]:
# %matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure()
lw = 2
plt.plot(fpr, tpr, color = 'darkorange', lw=lw,
         label = 'ROC curve (area = %0.f)' % roc_auc)
plt.plot([0 , 1], [0 , 1], color = 'navy',lw=lw, linestyle = '--')
plt.xlim([0.0 , 1.0])
plt.ylim([0.0 , 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Reciever Operating Characteristics')
plt.legend(loc="lower right")
plt.show()

In [ ]:
prob = model.predict_generator(generator=validation_generator)

y_true = validation_generator.classes
# print(y_true)

y_pred=prob>0.5
# print(y_pred)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
mat = confusion_matrix(y_true , y_pred)
#print(mat)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true , y_pred)

import seaborn as sns
import matplotlib.pyplot as plt

ax=plt.subplot()
sns.heatmap(cm , annot=True , ax=ax)

In [ ]:
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]

In [ ]:
# Sensitivity or true positive rate
TPR = TP/(TP+FN)
TPR_f = TPR*100
# Specificity or true negative rate
TNR = TN/(TN+FP)
TNR_f = TNR*100
# Precision or positive predictive value
PPV = TP/(TP+FP)
PPV_f = PPV*100
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)
ACC_f = ACC*100
#Recall
recall = (TP/(TP+FN))
recall_f = recall*100
#F1 Score
F1 = (2*((PPV*recall)/(PPV+recall)))
F1_f = F1*100


print("False Positive                                  : ",FP)
print("False Neagtive                                  : ",FN)
print("True Positive                                   : ",TP)
print("True Negative                                   : ",TN)
print("True Positive Rate (Sensitivity)                : %.2f "%TPR_f)
print("True Negative Rate (Specificity)                : %.2f "%TNR_f)
print("Precision                                       : %.2f "%PPV_f)
print("Accuracy for each class                         : %.2f "%ACC_f)
print("Recall                                          : %.2f "%recall_f)
print("F1 Score                                        : %.2f "%F1_f) 